## Goal:
Store the NBA draft data in a sql data table.

Import packages for reading in data from a csv, then loading it into our database.

In [1]:
import pandas as pd
import numpy as np
import getpass
import psycopg2
from psycopg2.extensions import adapt, register_adapter, AsIs

Create connection to database

In [2]:
mypasswd = getpass.getpass()
conn = psycopg2.connect(database = 'cs20_group4',
                              user = 'jpcp73',
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

········


Read in `NBA_draft_data` csv

In [3]:
data = pd.read_csv('..//Data/nba_draft_data.csv')

In [4]:
print(data.shape)
data.head()

(7755, 5)


,player_name,pick,school,team,year
0,Paul Arizin,NaN,Villanova,PHW,1950
1,Chuck Share,1.0,Bowling Green State,BOS,1950
2,Don Rehfeldt,2.0,Wisconsin,BLB,1950
3,Bob Cousy,3.0,Holy Cross,TRI,1950
4,Dick Schnittker,4.0,Ohio State,WSC,1950


Create table into which we will insert the data above

In [5]:
cursor = conn.cursor()

In [6]:
create_table = """
DROP TABLE IF EXISTS nba_draft_results;
CREATE TABLE nba_draft_results (
player_name varchar(100)
, pick numeric
, school varchar(250)
, team varchar(100)
, year numeric
)
"""

Commit newly created table to the database

In [7]:
cursor.execute(create_table)

In [8]:
conn.commit()

Reconnect to database for insertion of data

In [9]:
mypasswd = getpass.getpass()
conn = psycopg2.connect(database = 'cs20_group4',
                              user = 'jpcp73',
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

········


In [10]:
cursor = conn.cursor()

Write data to database

In [11]:
df = data.where(pd.notnull(data),None)

INSERT_SQL = "INSERT INTO nba_draft_results"
INSERT_SQL += "(player_name"
INSERT_SQL += ", pick"
INSERT_SQL += ", school"
INSERT_SQL += ", team"
INSERT_SQL += ", year) VALUES"
INSERT_SQL += "(%s,%s,%s,%s,%s)"

with conn, conn.cursor() as cursor:
    for row in df.itertuples(index=False, name=None):
        cursor.execute(INSERT_SQL,row)

Pull data from database to verify that everything was loaded correctly

In [13]:
test = pd.read_sql_query("""
SELECT
    *
FROM nba_draft_results
""", con = conn)
print(test.shape)
test.head()

(7755, 5)


,player_name,pick,school,team,year
0,Paul Arizin,NaN,Villanova,PHW,1950.0
1,Chuck Share,1.0,Bowling Green State,BOS,1950.0
2,Don Rehfeldt,2.0,Wisconsin,BLB,1950.0
3,Bob Cousy,3.0,Holy Cross,TRI,1950.0
4,Dick Schnittker,4.0,Ohio State,WSC,1950.0
